In [1]:
import pandas as pd
import os
import random
from tqdm import tqdm
import pickle
from sklearn.preprocessing import MinMaxScaler

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
merge_context_data = False
regenerate_context_data = True
sequence_augmentation = True
whole_session_context = False
model_test_run = True

PATH_TO_LOAD = '../data/04_Merged'
combined_context_path = '../data/05_Interaction_Sequences/context.csv'

window = 100 #seconds

base_path = '../datasets/sequential/'
augmentation_folder = 'aug/' if sequence_augmentation else 'non_aug/'
if model_test_run:
    augmentation_folder = 'test/aug/' if sequence_augmentation else 'test/non_aug/'

sequence_context_path = f'{base_path}{augmentation_folder}parameters/sequence_context.csv'
parameter_path = f'{base_path}{augmentation_folder}parameters/param.pkl'
train_session_path = f'{base_path}{augmentation_folder}parameters/train_sessions.pkl'
test_session_path = f'{base_path}{augmentation_folder}parameters/test_sessions.pkl'
train_dynamic_context_path = f'{base_path}{augmentation_folder}dynamic_context/train.csv'
test_dynamic_context_path = f'{base_path}{augmentation_folder}dynamic_context/test.csv'
train_static_context_path = f'{base_path}{augmentation_folder}static_context/train.csv'
test_static_context_path = f'{base_path}{augmentation_folder}static_context/test.csv'

In [3]:
all_columns = ['index', 'avg_irradiation', 'steering_speed', 'temperature_out', 'hour',
       'month', 'odometer', 'light_sensor_rear', 'light_sensor_front',
       'temperature_in', 'KBI_speed', 'soc', 'ESP_speed', 'latitude',
       'longitude', 'seatbelt_codriver', 'seatbelt_rear_l', 'seatbelt_rear_m',
       'seatbelt_rear_r', 'CHA_ESP_drive_mode', 'CHA_MO_drive_mode',
       'rain_sensor', 'street_category', 'kickdown', 'altitude',
       'driving_program', 'datetime', 'session', 'Label', 'ID',
       'FunctionValue', 'domain', 'BeginTime', 'time_second',
       'distance_driven', 'ts_normalized', 'weekday']

selected = [ 'avg_irradiation', 'steering_speed', 'temperature_out', 'hour',
       'month', 'light_sensor_rear', 'light_sensor_front',
       'temperature_in', 'KBI_speed', 'soc', 'latitude',
       'longitude', 'seatbelt_codriver', 'seatbelt_rear_l',
       'seatbelt_rear_r', 'street_category', 'altitude',
       'datetime', 'session', 'time_second',
       'distance_driven', 'weekday'
]

bad_quality = ['CHA_ESP_drive_mode', 
             'CHA_MO_drive_mode',
             'rain_sensor',
             'kickdown',
             'ESP_speed',
             'seatbelt_rear_m',
            'driving_program',
            'ts_normalized'
             ]

dynamic_context_var = ['avg_irradiation', 'steering_speed', 'temperature_out', 
                       'light_sensor_rear', 'light_sensor_front', 
                       'temperature_in', 'KBI_speed', 'soc', 'latitude',
                       'longitude',  'street_category', 'altitude','time_second',
                       'distance_driven']
static_context_var = ['car_id', 'month', 'weekday','hour', 'seatbelt_codriver', 'seatbelt_rear_l',
                       'seatbelt_rear_r',]

In [4]:
def load_context(vehicle):
    df = pd.read_csv(os.path.join(PATH_TO_LOAD, vehicle + "_merged.csv"), parse_dates=['datetime'], low_memory=False)
    context_lists = dynamic_context_var + static_context_var + ['session', 'datetime']
    context_lists.remove('car_id')
    df_filt = df[context_lists]
    df_filt = df_filt.dropna(subset=['KBI_speed'])
    df_filt_sort = df_filt.sort_values(by=['session','datetime'])
    return df_filt_sort

vehicles = ['SEB880','SEB882','SEB883','SEB885','SEB888','SEB889']
context_data = pd.DataFrame()

if merge_context_data == True:
    for vehicle in tqdm(vehicles):
        context_curr = load_context(vehicle)
        context_curr['car_id'] = vehicle
        context_data = pd.concat([context_data, context_curr], axis=0)
    context_data.to_csv(combined_context_path)

if regenerate_context_data == True:
    context_data = pd.read_csv(combined_context_path, parse_dates=['datetime'], index_col=0)
    vehicle_list = context_data.car_id.unique().tolist()
    vehicle_dict = {vehicle: random.randint(1, 50) for vehicle in vehicle_list}
    context_data['car_id'] = context_data['car_id'].map(vehicle_dict)
    context_data = context_data.sort_values(by=['session','datetime'])
    context_data['session'] = context_data['session'].astype(int)
    # static_context_var.append('session')
    # static_context = context_data[static_context_var].drop_duplicates(subset=['car_id', 'session'])
    dynamic_context_var.extend(['session', 'datetime'])
    context_data = context_data[dynamic_context_var + static_context_var]
    context_var = [item for item in dynamic_context_var + static_context_var if item not in ['session', 'datetime']]
    context_data = context_data.groupby(['session', 'datetime'])[context_var].mean().reset_index()

    if model_test_run:
        with open(train_session_path, 'rb') as pickle_file:
            train_sessions = pickle.load(pickle_file)

        with open(test_session_path, 'rb') as pickle_file:
            test_sessions = pickle.load(pickle_file)
            
        additional_columns = ['session_id']
        context_data = context_data[context_data['session'].isin(train_sessions + test_sessions)]

In [5]:
selected_sequence = pd.read_csv(sequence_context_path, parse_dates=['datetime'], index_col=0).reset_index()
selected_sequence['session'] = selected_sequence['session'].astype(int)

min_datetime_indices = selected_sequence.groupby('session')['datetime'].idxmin()
selected_sequence = selected_sequence.drop(min_datetime_indices)
selected_sequence.reset_index(drop=True, inplace=True)
context_data = context_data[context_data['session'].isin(selected_sequence.session.unique().tolist())]

training_sequence_context = pd.DataFrame()
selected_dfs = []
for session in tqdm(selected_sequence['session'].unique().tolist()):
    selected_sequence_curr = selected_sequence[selected_sequence['session']==session]
    context_data_curr = context_data[context_data['session']==session]
    context_data_curr = context_data_curr[context_data_curr['datetime']<=selected_sequence_curr['datetime'].max()]
    selected_dfs.append(context_data_curr)
    # training_sequence_context = pd.concat([training_sequence_context,context_data_curr], axis=0)

training_sequence_context = pd.concat(selected_dfs, axis=0)

100%|██████████| 7/7 [00:00<00:00, 248.58it/s]


In [2]:

import torch

# Example tensors
tensor1 = torch.tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
tensor2 = torch.tensor([8, 8])

# Concatenate along dimension 0 (rows)
concatenated_tensor = torch.cat((tensor1, tensor2), dim=0)

print("Tensor 1:")
print(tensor1)

print("\nTensor 2:")
print(tensor2)

print("\nConcatenated Tensor along dimension 0:")
print(concatenated_tensor)


Tensor 1:
tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

Tensor 2:
tensor([8, 8])

Concatenated Tensor along dimension 0:
tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 8, 8])


In [5]:
tensor2.size()

torch.Size([2])

In [10]:
import torch

# Create a PyTorch tensor
tensor = torch.tensor([[-6.8802e-03, -4.1273e-03,  1.3144e-02, -3.5430e-03, -2.4565e-03,
         -3.8868e-03, -3.3703e-03, -7.3480e-03, -5.5187e-05],
        [-6.8802e-03, -4.1273e-03,  1.3144e-02, -3.5430e-03, -2.4565e-03,
         -3.8868e-03, -3.3703e-03, -7.3480e-03, -5.5187e-05],
        [-6.8802e-03, -4.1273e-03,  1.3144e-02, -3.5430e-03, -2.4565e-03,
         -3.8868e-03, -3.3703e-03, -7.3480e-03, -5.5187e-05],
        [-6.8802e-03, -4.1273e-03,  1.3144e-02, -3.5430e-03, -2.4565e-03,
         -3.8868e-03, -3.3703e-03, -7.3480e-03, -5.5187e-05],
        [-6.8802e-03, -4.1273e-03,  1.3144e-02, -3.5430e-03, -2.4565e-03,
         -3.8868e-03, -3.3703e-03, -7.3480e-03, -5.5187e-05],
        [-6.8802e-03, -4.1273e-03,  1.3144e-02, -3.5430e-03, -2.4565e-03,
         -3.8868e-03, -3.3703e-03, -7.3480e-03, -5.5187e-05],
        [-6.8802e-03, -4.1273e-03,  1.3144e-02, -3.5430e-03, -2.4565e-03,
         -3.8868e-03, -3.3703e-03, -7.3480e-03, -5.5187e-05],
        [-6.8802e-03, -4.1273e-03,  1.3144e-02, -3.5430e-03, -2.4565e-03,
         -3.8868e-03, -3.3703e-03, -7.3480e-03, -5.5187e-05],
        [-6.8802e-03, -4.1273e-03,  1.3144e-02, -3.5430e-03, -2.4565e-03,
         -3.8868e-03, -3.3703e-03, -7.3480e-03, -5.5187e-05],
        [-6.8802e-03, -4.1273e-03,  1.3144e-02, -3.5430e-03, -2.4565e-03,
         -3.8868e-03, -3.3703e-03, -7.3480e-03, -5.5187e-05],
        [-6.8802e-03, -4.1273e-03,  1.3144e-02, -3.5430e-03, -2.4565e-03,
         -3.8868e-03, -3.3703e-03, -7.3480e-03, -5.5187e-05],
        [-6.8802e-03, -4.1273e-03,  1.3144e-02, -3.5430e-03, -2.4565e-03,
         -3.8868e-03, -3.3703e-03, -7.3480e-03, -5.5187e-05],
        [-6.8802e-03, -4.1273e-03,  1.3144e-02, -3.5430e-03, -2.4565e-03,
         -3.8868e-03, -3.3703e-03, -7.3480e-03, -5.5187e-05],
        [-6.8802e-03, -4.1273e-03,  1.3144e-02, -3.5430e-03, -2.4565e-03,
         -3.8868e-03, -3.3703e-03, -7.3480e-03, -5.5187e-05],
        [-6.8802e-03, -4.1273e-03,  1.3144e-02, -3.5430e-03, -2.4565e-03,
         -3.8868e-03, -3.3703e-03, -7.3480e-03, -5.5187e-05],
        [-6.8802e-03, -4.1273e-03,  1.3144e-02, -3.5430e-03, -2.4565e-03,
         -3.8868e-03, -3.3703e-03, -7.3480e-03, -5.5187e-05]])

# Specify the number of top elements to retrieve
k = 3

# Use torch.topk to get the indices of the top k values
top_values, top_indices = torch.topk(tensor, k)

print("Original Tensor:")
print(tensor)

print("\nTop {} Values:".format(k))
print(top_values)
import numpy as np
print("\nIndices of Top {} Values:".format(k))
print(np.mean(top_indices.numpy()))


Original Tensor:
tensor([4, 8, 2, 7, 1, 9, 3])

Top 3 Values:
tensor([9, 8, 7])

Indices of Top 3 Values:
3.0


In [6]:
# testing_sessions = [16, 25]
# selected_sequence = selected_sequence[selected_sequence['session'].isin(testing_sessions)]
# training_sequence_context = training_sequence_context[training_sequence_context['session'].isin(testing_sessions)]

training_sequence_context_augmented = pd.DataFrame()
session_id = 0
if sequence_augmentation == True:
    # for session in tqdm(selected_sequence['session'].unique().tolist()):
    #     selected_sequence_curr = selected_sequence[selected_sequence['session']==session].reset_index()
    #     context_curr = training_sequence_context[training_sequence_context['session']==session].reset_index()
    #     for i in range(len(selected_sequence_curr)-1, -1, -1):
    #         context_filt_curr = training_sequence_context[
    #             (training_sequence_context['datetime'] <= selected_sequence_curr.loc[i, 'datetime'])].copy()
    #         if whole_session_context == False:
    #             context_filt_curr = context_filt_curr.tail(window)
    #         # context_filt_curr.loc[context_filt_curr.index, 'session_id'] = session_id
    #         context_filt_curr['session_id'] = session_id
    #         training_sequence_context_augmented = pd.concat([training_sequence_context_augmented, context_filt_curr], axis=0)
    #         session_id += 1
    # context_data = training_sequence_context_augmented
    grouped_selected_sequence = selected_sequence.groupby('session')
    augmented_frames = []

    for session, selected_sequence_curr in tqdm(grouped_selected_sequence):
        for i, row in selected_sequence_curr.iloc[::-1].iterrows():
            context_filt_curr = training_sequence_context[
                (training_sequence_context['session'] == session) &
                (training_sequence_context['datetime'] <= row['datetime'])
            ].copy()
            if not whole_session_context:
                context_filt_curr = context_filt_curr.tail(window)
            context_filt_curr['session_id'] = session_id

            augmented_frames.append(context_filt_curr)
            session_id += 1
    training_sequence_context_augmented = pd.concat(augmented_frames, axis=0)
    context_data = training_sequence_context_augmented

else:
    # if sequence_augmentation is set to false
    if whole_session_context == True:
        context_data = training_sequence_context
    else:
        context_data = training_sequence_context.groupby('session').tail(window)

100%|██████████| 7/7 [00:00<00:00, 114.48it/s]


In [7]:
# in the dynamic context there are some windows with time instances are less than 100 handle somehow
with open(train_session_path, 'rb') as pickle_file:
    train_sessions = pickle.load(pickle_file)

with open(test_session_path, 'rb') as pickle_file:
    test_sessions = pickle.load(pickle_file)
    
additional_columns = ['session_id']
context_data = context_data[context_data['session'].isin(train_sessions + test_sessions)]
# static_context = static_context[static_context['session'].isin(train_sessions + test_sessions)]
static_context = context_data[static_context_var + additional_columns + ['session']]
static_context = static_context.drop_duplicates()
dynamic_context = context_data[dynamic_context_var + additional_columns]

train_dynamic_context = dynamic_context[dynamic_context['session'].isin(train_sessions)]
test_dynamic_context = dynamic_context[dynamic_context['session'].isin(test_sessions)]
train_static_context = static_context[static_context['session'].isin(train_sessions)]
test_static_context = static_context[static_context['session'].isin(test_sessions)]

if sequence_augmentation:
    train_dynamic_context['session_id'] = train_dynamic_context.groupby('session_id').ngroup()
    test_dynamic_context['session_id'] = test_dynamic_context.groupby('session_id').ngroup()
else:
    train_dynamic_context['session_id'] = train_dynamic_context.groupby('session').ngroup()
    test_dynamic_context['session_id'] = test_dynamic_context.groupby('session').ngroup()

train_dynamic_context = train_dynamic_context.drop(columns=['session'])
test_dynamic_context = test_dynamic_context.drop(columns=['session'])

dynamic_context_to_normalize = [col for col in train_dynamic_context.columns if col not in ['session_ids', 'datetime', 'session_id', 'session']]
scaler_dynamic_context = MinMaxScaler()
scaler_dynamic_context.fit(train_dynamic_context[dynamic_context_to_normalize])
train_dynamic_context[dynamic_context_to_normalize] = scaler_dynamic_context.transform(train_dynamic_context[dynamic_context_to_normalize])
test_dynamic_context[dynamic_context_to_normalize] = scaler_dynamic_context.transform(test_dynamic_context[dynamic_context_to_normalize])

train_static_context['session'] = range(len(train_static_context))
test_static_context['session'] = range(len(test_static_context))

# static_context_to_normalize = [col for col in train_static_context.columns if col not in ['session_ids', 'datetime', 'session_id', 'session']]
# scaler_static_context = MinMaxScaler()
# scaler_static_context.fit(train_static_context[static_context_to_normalize])
# train_static_context[static_context_to_normalize] = scaler_static_context.transform(train_static_context[static_context_to_normalize])
# test_static_context[static_context_to_normalize] = scaler_static_context.transform(test_static_context[static_context_to_normalize])

train_static_context = train_static_context.drop(columns=['session_id'])
test_static_context = test_static_context.drop(columns=['session_id'])

train_dynamic_context.to_csv(train_dynamic_context_path, index=False)
test_dynamic_context.to_csv(test_dynamic_context_path, index=False)

train_static_context.to_csv(train_static_context_path, index=False)
test_static_context.to_csv(test_static_context_path, index=False)

/var/folders/n8/1ntcg5gj7nvf5wh_s2w95yh80000gn/T/ipykernel_22263/3409487816.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_dynamic_context['session_id'] = train_dynamic_context.groupby('session_id').ngroup()
/var/folders/n8/1ntcg5gj7nvf5wh_s2w95yh80000gn/T/ipykernel_22263/3409487816.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dynamic_context['session_id'] = test_dynamic_context.groupby('session_id').ngroup()
/var/folders/n8/1ntcg5gj7nvf5wh_s2w95yh80000gn/T/ipykernel_22263/340948781

In [8]:
print(len(train_static_context.session.unique().tolist()), len(test_static_context.session.unique().tolist()))
print(len(train_dynamic_context.session_id.unique().tolist()), len(test_dynamic_context.session_id.unique().tolist()))

18 2
18 2


In [9]:
train_static_context

,car_id,month,weekday,seatbelt_codriver,seatbelt_rear_l,seatbelt_rear_r,session
969165,11.0,6.0,5.0,0.0,0.0,0.0,0
968779,11.0,6.0,5.0,0.0,0.0,0.0,1
968770,11.0,6.0,5.0,0.0,0.0,0.0,2
968558,11.0,6.0,5.0,0.0,0.0,0.0,3
967632,11.0,6.0,5.0,0.0,0.0,0.0,4
1045587,11.0,6.0,3.0,0.0,0.0,0.0,5
1042378,11.0,6.0,3.0,0.0,0.0,0.0,6
1039701,11.0,6.0,3.0,0.0,0.0,0.0,7
2248797,26.0,10.0,1.0,0.0,0.0,0.0,8
2248735,26.0,10.0,1.0,0.0,0.0,0.0,9
